# Hooks

Hooks allow you to use state and other React features without writing an ES6 class.

Here is an example:

In [ ]:
function Example() {
  const [count, setCount] = useState(0);
  return (
    <div>
      <p>You clicked {count} times</p>
      <button onClick={() => setCount(count + 1)}>
        Click me
      </button>
    </div>
  );
}

How would you do this using the ES6 class?

In [ ]:
class Example extend React.Component{
    constructor(props){
        super(props);
        this.state = {count: 0};
        this.handler = this.handler.bind(this);
    }
    handler(e){
        this.setState({count: this.state.count + 1});
    }
    render(){
        return(
            <div>
                <p>You clicked {this.state.count} times</p>
                <button onClick={this.handler}>
                    Click Me
                </button>
            </div>
        );
    }
}

See how easy it? Let's dive in:

## State Hook

Lets look back at the example above:

In [ ]:
function Example() {
  const [count, setCount] = useState(0);
  return (
    <div>
      <p>You clicked {count} times</p>
      <button onClick={() => setCount(count + 1)}>
        Click me
      </button>
    </div>
  );
}

`useState()` is a ***Hook***. This is called in a function component to add state to it. But `useState()` returns a pair:  
1. The CURRENT state value. (In this case, `count`)
2. A function that will let you update the state. (In this case, `setCount()`)  
You can call the returned function from an event handler, or elsewhere. It's similar to `setState`, BUT it does NOT merge the old and new state together.  

The argument in `useState(arg)`,  is the INITIAL state. In this example, the initial state is 0. This state can be any object you want it to be, like string, array, etc. This initial state argument is only used during the first render.

Here is an example of declaring multiple:

In [ ]:
function ExampleWithManyStates() {
  // Declare multiple state variables!
  const [age, setAge] = useState(42);
  const [fruit, setFruit] = useState('banana');
  const [todos, setTodos] = useState([{ text: 'Learn Hooks' }]);
  // ...
}

Note that the variable values and function values returned from `useState` can be named anything. But for ease of use, it is conventional to use: `[xyz, setXyz]`

## Effect Hook

The Effect Hook, `useEffect`, lets us perform side effects from a function component. It's like `componentDidMount`, `componentDidUpdate`, and `componentWeillUnmount` in react classes.

Here is an example that sets the document title after React updates the DOM:

In [ ]:
import React, { useState, useEffect } from 'react';

function Example() {
  const [count, setCount] = useState(0);

  // Similar to componentDidMount and componentDidUpdate:
  useEffect(() => {
    // Update the document title using the browser API
    document.title = `You clicked ${count} times`;
  });

  return (
    <div>
      <p>You clicked {count} times</p>
      <button onClick={() => setCount(count + 1)}>
        Click me
      </button>
    </div>
  );
}

Wehn using `useEffect`, the "effect" function runs after "flushing" changes to the DOM. So React runs the effects after every render, the first render included.

You can optionally specify how to "clean up" afterwards by returning a function. Here is an example:

In [ ]:
import React, { useState, useEffect } from 'react';

function FriendStatus(props) {
  const [isOnline, setIsOnline] = useState(null);

  function handleStatusChange(status) {
    setIsOnline(status.isOnline);
  }

  useEffect(() => {
    ChatAPI.subscribeToFriendStatus(props.friend.id, handleStatusChange);
    return () => {
      ChatAPI.unsubscribeFromFriendStatus(props.friend.id, handleStatusChange);
    };
  });

  if (isOnline === null) {
    return 'Loading...';
  }
  return isOnline ? 'Online' : 'Offline';
}

Try to understand what this does

## Rules of Hooks

Hooks are JS functions, but impose two additional rules:
* Only call Hooks at top level. Don't call Hooks inside loops, conditions, or nested functions.
* Only call Hooks from React function components. Don't call Hooks from regular JavaScript functions.  


But you can also place hooks in custom Hooks.

## Building Your Own Hooks

Sometimes you want to reuse some stateful logic between components. You would normaly use ***higher-order components*** and ***render props***. (We may not have used these yet, we will later). But now we can use custom Hooks to let us do this, but without adding more components to the "tree".

Let's say we want to resuse the `FriendStatus` component's logic into another component. Let's try it by using a custom Hook:

In [ ]:
import React, { useState, useEffect } from 'react';

function useFriendStatus(friendID) {
  const [isOnline, setIsOnline] = useState(null);

  function handleStatusChange(status) {
    setIsOnline(status.isOnline);
  }

  useEffect(() => {
    ChatAPI.subscribeToFriendStatus(friendID, handleStatusChange);
    return () => {
      ChatAPI.unsubscribeFromFriendStatus(friendID, handleStatusChange);
    };
  });

  return isOnline;
}

This looks almost identical to the one above. But notice how we pass in `friendID` rather than `props`. But now we can use it in multiple components:

In [ ]:
function FriendStatus(props) {
  const isOnline = useFriendStatus(props.friend.id);

  if (isOnline === null) {
    return 'Loading...';
  }
  return isOnline ? 'Online' : 'Offline';
}

In [ ]:
function FriendListItem(props) {
  const isOnline = useFriendStatus(props.friend.id);

  return (
    <li style={{ color: isOnline ? 'green' : 'black' }}>
      {props.friend.name}
    </li>
  );
}

But every single call to a Hook has an isolated state. So we can use the same custom Hook twice in one component. By convention, if a function's name starts with `"use"`, and calls other Hooks, then it is labeled a custom Hook.

## Other Hooks

There are other not commonly used built-in Hooks like:

### `useContext`

This lets you subscribe to React context without introducing nesting:

In [ ]:
function Example() {
  const locale = useContext(LocaleContext);
  const theme = useContext(ThemeContext);
  // ...
}

### useReducer

Lets you manage local state of complex components with a reducer:

In [ ]:
function Todos() {
  const [todos, dispatch] = useReducer(todosReducer);
  // ...